In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import time
import re


import torch as T



import torch.nn as nn
import torch.nn.functional as F


# from model2 import Model


from data_util import config
from data_util import bert_data as data
from data_util.bert_batcher import Batcher
from data_util.bert_data import Vocab
from write_result import *
# from data_util import data
# from data_util.batcher import Batcher
# from data_util.data import Vocab


from train_bert_util import *
from torch.distributions import Categorical
from rouge import Rouge
from numpy import random
import argparse
import torchsnooper
import logging
# transformers_logger = logging.getLogger("transformers.tokenization_utils")
# transformers_logger.setLevel(logging.ERROR)
# transformers_logger.disabled = True

# # -------- Test Packages -------
# from bert_enc_beam_search import *
# import shutil
# from tensorboardX import SummaryWriter
# from nltk.translate.bleu_score import corpus_bleu

# # from pytorch_pretrained_bert import BertModel
# from transformers import BertModel, BertTokenizer 
# from transformers import TransfoXLTokenizer, TransfoXLModel, TransfoXLConfig

config.batch_size = 1
config.emb_dim = 768
config.vocab_size = 50000

# config.hidden_dim = 512
config.hidden_dim = 768

config.max_enc_steps = 500
config.lr = 0.001 # 0.001

# config.max_dec_steps = 60		#99% of the titles are within length 15
# config.min_dec_steps= 8
config.ber_layer = 11 # last_layer [0~11]
# config.key_attention = False
# help(config)

I0319 21:33:01.254710 139846209922880 file_utils.py:35] PyTorch version 1.3.1 available.


We have  30526 bert tokens now
We have added 3 XL tokens


In [2]:
info_str = ''
for a in dir(config):
    if type(getattr(config, a)) in [str,int,float,bool] \
    and 'path' not in str(a) \
    and '__' not in str(a) \
    and 'info' not in str(a):

        info_str += '## %s : %s\n'%(a,getattr(config, a))

# [print(a,getattr(config, a)) for a in dir(config)
# if type(getattr(config, a)) in [str,int,float]
#  and 'path' not in str(a)
#  and '__' not in str(a)
#  and 'info' not in str(a)
# ]
print(info_str)

## batch_size : 1
## beam_size : 16
## ber_layer : 11
## data_type : Cameras_new8
## emb_dim : 768
## emb_grad : False
## eps : 1e-12
## gound_truth_prob : 0.1
## hidden_dim : 768
## intra_decoder : True
## intra_encoder : True
## key_attention : False
## keywords : POS_FOP_keywords
## loggerName : Text-Summary
## lr : 0.001
## max_dec_steps : 50
## max_enc_steps : 500
## max_epochs : 100
## max_iterations : 500000
## max_key_num : 8
## min_dec_steps : 4
## rand_unif_init_mag : 0.02
## trunc_norm_init_std : 0.0001
## vocab_size : 50000
## word_emb_type : word2Vec



# Logger

In [3]:
from datetime import datetime as dt

def getLogger(loggerName, loggerPath):
    # 設置logger
    logger = logging.getLogger(loggerName)  # 不加名稱設置root logger
    logger.setLevel(logging.DEBUG)
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s: - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')
    logging.Filter(loggerName)

    # 使用FileHandler輸出到文件
    directory = os.path.dirname(loggerPath)
    if not os.path.exists(directory):
        os.makedirs(directory)
    fh = logging.FileHandler(loggerPath)

    fh.setLevel(logging.DEBUG)
    fh.setFormatter(formatter)

    # 使用StreamHandler輸出到屏幕
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    ch.setFormatter(formatter)
    # 添加兩個Handler
    logger.addHandler(ch)
    logger.addHandler(fh)
    # Handler只啟動一次
    # 設置logger
    logger.info(u'logger已啟動')
    return logger

def removeLogger(logger):
    logger.info(u'logger已關閉')
    handlers = logger.handlers[:]
    for handler in handlers:
        handler.close()
        logger.removeHandler(handler)

# View batch data

In [4]:
def test_batch():
    vocab = Vocab(config.vocab_path, config.vocab_size)
    batcher = Batcher(config.train_data_path, vocab, mode='train',
                           batch_size=config.batch_size, single_pass=False)
    batch = batcher.next_batch()
    # with torchsnooper.snoop():
    while batch is not None:
        example_list = batch.example_list
        for ex in example_list:
            r = str(ex.original_review)
            s = str(ex.original_summary)
            k = str(ex.key_words)
            sent = ex.original_summary_sents
#             print("original_review_sents:", r)
#             print("original_summary_sents : ", s)
            print("key_words : ", k)
#             print('------------------------------------------------------------\n')
        batch = batcher.next_batch()        
        break
test_batch()

key_words :  ['mode', 'manual', 'difference', 'shock', 'camera', 'prehistoric', 'thing', 'good']


# Get Bin Information

In [5]:
 with open(config.bin_info,'r',encoding='utf-8') as f:
    lines = f.readlines()
    [print(line) for line in lines]
    train_num = int(lines[0].split(":")[1])
    test_num = int(lines[1].split(":")[1])
    val_num = int(lines[2].split(":")[1])
    # f.write("train : %s\n"%(len(flit_key_train_df)))
    # f.write("test : %s\n"%(len(flit_key_test_df)))
    # f.write("valid : %s\n"%(len(flit_key_valid_df)))


train : 33574

test : 4196

valid : 4196



# View model summary
#### 只有torchsummaryX成功
#### 日後將以此模擬呈現結構

In [6]:
# from torchsummaryX import summary
# from model2 import Encoder,Model
# device = T.device("cuda" if T.cuda.is_available() else "cpu") # PyTorch v0.4.0
# encoder = Encoder().to(device)    

# vocab = Vocab(config.vocab_path, config.vocab_size)
# batcher = Batcher(config.train_data_path, vocab, mode='train',
#                        batch_size=config.batch_size, single_pass=False)
# batch = batcher.next_batch()
# enc_batch, enc_lens, enc_padding_mask, enc_key_batch, enc_key_lens, enc_key_padding_mask, enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)
     
# # bert_model = TransfoXLModel(xl_config) # 更改參數以傳入TransfoXLModel
# # bert_model = get_cuda(TransfoXLModel.from_pretrained('transfo-xl-wt103'))
# # all_hidden_states, _ = bert_model(enc_batch)[-2:]
   
# # summary(encoder, enc_batch, enc_padding_mask, enc_lens) # encoder summary (ok)

# print('test success')
# # model = Model(False,'word2Vec',vocab)
# # enc_out, enc_hidden = model.encoder(enc_batch, enc_padding_mask, enc_lens) # encoder summary (ok)
# # print(enc_hidden.shape)

# Summary Decoder

In [7]:
# from torchsummaryX import summary
# from model2 import Decoder,Model
# from train_bert_util import *
# device = T.device("cuda" if T.cuda.is_available() else "cpu") # PyTorch v0.4.0
# # decoder = Decoder().to(device)    

# model = Model(False,'word2Vec',vocab)
# # vocab = Vocab(config.vocab_path, config.vocab_size)
# # batcher = Batcher(config.train_data_path, vocab, mode='train',
# #                        batch_size=config.batch_size, single_pass=False)
# # batch = batcher.next_batch()
# # enc_batch, enc_lens, enc_padding_mask, enc_key_batch, enc_key_lens, enc_key_padding_mask, enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)
# # enc_batch = model.embeds(enc_batch) #Get embeddings for encoder input
# # enc_out, enc_hidden = model.encoder(enc_batch, enc_lens)

# # encoder = Encoder().to(device)   
# vocab = Vocab(config.vocab_path, config.vocab_size)
# batcher = Batcher(config.train_data_path, vocab, mode='train',
#                        batch_size=config.batch_size, single_pass=False)
# batch = batcher.next_batch()
# enc_batch, enc_lens, enc_padding_mask, enc_key_batch, enc_key_lens, enc_key_padding_mask, enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)
# # print('enc_batch',enc_batch.shape)
# # print(enc_batch[0][:5])

# enc_out, enc_hidden = model.encoder(enc_batch, enc_padding_mask, enc_lens) # encoder summary (ok)
# # print('enc_out',enc_out.shape)
# # print('enc_hidden',enc_hidden.shape)
# # print('encoder success')  

# start_decoding = vocab.word2id(bert_data.START_DECODING) # start_decoding = 30524
# stop_decoding = vocab.word2id(bert_data.STOP_DECODING) # stop_decoding = 30525
    
# # train_batch_MLE
# dec_batch, max_dec_len, dec_lens, target_batch = get_dec_data(batch)                        #Get input and target batchs for training decoder
# step_losses = []
# # s_t = (enc_hidden[0], enc_hidden[1])                                                        #Decoder hidden states
# s_t = (enc_hidden,enc_hidden)
# # x_t 為decoder每一個time step 的batch input
# x_t = get_cuda(T.LongTensor(len(enc_out)).fill_(start_decoding)) # initial batch decode word                            #Input to the decoder
# prev_s = None                                                                               #Used for intra-decoder attention (section 2.2 in DEEP REINFORCED MODEL - https://arxiv.org/pdf/1705.04304.pdf)
# sum_temporal_srcs = None     
# unk_id = vocab.word2id(data.PAD_TOKEN)

# # print('x_t',x_t.shape)
# # print(vocab.vocab)     
# # print('enc_hidden',enc_hidden.shape)

# # s_t = s_t[:,0,:]
# for t in range(min(max_dec_len, config.max_dec_steps)):
#     print('decoder time step %s'%t)
# #     T.rand(len(enc_out)) tensor([0.6797, 0.7603])
#     use_gound_truth = get_cuda((T.rand(len(enc_out)) > 0.25)).long()                        #Probabilities indicating whether to use ground truth labels instead of previous decoded tokens
#     # use_gound_truth * dec_batch[:, t] : 為ground true time step token
#     # (1 - use_gound_truth) * x_t : 為previous time step token
#     x_t = use_gound_truth * dec_batch[:, t] + (1 - use_gound_truth) * x_t                   #Select decoder input based on use_ground_truth probabilities   
#     x_t, _ = model.encoder(x_t, None, None) # encoder summary (ok)
# #     enc_key_batch = model.embeds(enc_key_batch)  

#     # use the output vector corresponding to [CLS] to initialize the hidden state and cell state of LSTM decoder
# #     print('s_t1',s_t[0].shape);print('s_t1',s_t[1].shape)
    
#     final_dist, s_t, ct_e, sum_temporal_srcs, prev_s = model.decoder(
#     x_t, s_t, enc_out, enc_padding_mask, context, 
#     extra_zeros,enc_batch_extend_vocab,sum_temporal_srcs, prev_s, 
#     None, None)    
# #     print('s_t2',s_t[0].shape);print('s_t2',s_t[1].shape)
#     target = target_batch[:, t]
#     log_probs = T.log(final_dist + config.eps)
#     step_loss = F.nll_loss(log_probs, target, reduction="none", ignore_index=unk_id)
#     step_losses.append(step_loss)
# #     x_t = T.multinomial(final_dist,1).squeeze()  # Sample words from final distribution which can be used as input in next time step
    
# #     is_oov = (x_t >= config.vocab_size).long()  # Mask indicating whether sampled word is OOV
# #     x_t = (1 - is_oov) * x_t.detach() + (is_oov) * unk_id  # Replace OOVs with [UNK] token
  
# #     print('s_t',s_t.shape)
#     decoder_summary = summary(model.decoder, x_t, s_t, enc_out, 
#                               enc_padding_mask, context, extra_zeros,
#                               enc_batch_extend_vocab, 
#                               sum_temporal_srcs, prev_s,None, 
#                               enc_key_lens) # encoder summary
#     print('finish inner loop'); print('-------------------------------------------------\n')
#     break
# # decoder_summary

# Train object

In [8]:
# from torch.autograd import Variable
# import torch
# def write_enc_graph():
#     encoder_writer = SummaryWriter('runs/Pointer-Generator/word2Vec/Encoder')
#     device = T.device("cuda" if T.cuda.is_available() else "cpu") # PyTorch v0.4.0
#     encoder = Encoder().to(device) 

#     vocab = Vocab(config.vocab_path, config.vocab_size)
#     batcher = Batcher(config.train_data_path, vocab, mode='train',
#                            batch_size=config.batch_size, single_pass=False)
#     batch = batcher.next_batch()
#     enc_batch, enc_lens, enc_padding_mask, enc_key_batch, enc_key_lens, enc_key_padding_mask, enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)
#     enc_batch = Model(False,'word2Vec',vocab).embeds(enc_batch) #Get embeddings for encoder input

# #     enc_batch = Variable(torch.rand(enc_batch.shape)).to(device) 
#     enc_lens = torch.from_numpy(enc_lens).to(device) 

#     encoder_writer.add_graph(encoder, (enc_batch, enc_lens), verbose=True)
#     encoder_writer.close()

# def write_dec_graph():
#     decoder_writer = SummaryWriter('runs/Pointer-Generator/word2Vec/Decoder')
#     device = T.device("cuda" if T.cuda.is_available() else "cpu") # PyTorch v0.4.0
#     # decoder = Decoder().to(device)    

#     vocab = Vocab(config.vocab_path, config.vocab_size)
#     model = Model(False,'word2Vec',vocab)
    
#     batcher = Batcher(config.train_data_path, vocab, mode='train',
#                            batch_size=config.batch_size, single_pass=False)
#     batch = batcher.next_batch()
#     enc_batch, enc_lens, enc_padding_mask, enc_key_batch, enc_key_lens, enc_key_padding_mask, enc_batch_extend_vocab, extra_zeros, context = get_enc_data(batch)
# #     enc_batch = model.embeds(enc_batch) #Get embeddings for encoder input
#     enc_out, enc_hidden = model.encoder(enc_batch, enc_lens)

#     # train_batch_MLE
#     dec_batch, max_dec_len, dec_lens, target_batch = get_dec_data(batch)                        #Get input and target batchs for training decoder
#     step_losses = []
#     s_t = (enc_hidden[0], enc_hidden[1])                                                        #Decoder hidden states
#     # x_t 為decoder每一個time step 的batch input
#     x_t = get_cuda(T.LongTensor(len(enc_out)).fill_(2))                             #Input to the decoder
#     prev_s = None                                                                               #Used for intra-decoder attention (section 2.2 in DEEP REINFORCED MODEL - https://arxiv.org/pdf/1705.04304.pdf)
#     sum_temporal_srcs = None     


#     for t in range(min(max_dec_len, config.max_dec_steps)):
#         use_gound_truth = get_cuda((T.rand(len(enc_out)) > 0.25)).long()                        #Probabilities indicating whether to use ground truth labels instead of previous decoded tokens
#         # use_gound_truth * dec_batch[:, t] : 為ground true time step token
#         # (1 - use_gound_truth) * x_t : 為previous time step token
#         if t == 0 :temp_batch = dec_batch[:, t]
#         x_t = use_gound_truth * temp_batch + (1 - use_gound_truth) * x_t                   #Select decoder input based on use_ground_truth probabilities
# #         x_t = model.embeds(x_t)
#     #     final_dist, s_t, ct_e, sum_temporal_srcs, prev_s = model.decoder(x_t, s_t, enc_out, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, sum_temporal_srcs, prev_s)
#         final_dist, s_t, ct_e, sum_temporal_srcs, prev_s = model.decoder(
#         x_t, s_t, enc_out, enc_padding_mask,context, 
#         extra_zeros,enc_batch_extend_vocab,sum_temporal_srcs, prev_s, 
#         enc_key_batch, enc_key_lens)        


#         #         decoder_summary = summary(model.decoder, x_t, s_t, enc_out, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, sum_temporal_srcs, prev_s,enc_key_batch, enc_key_lens) # encoder summary
# #         x_t = Variable(torch.rand(x_t.shape)).to(device) 
#         #             s_t = Variable(torch.rand(s_t.shape)).to(device)
# #         enc_out = Variable(torch.rand(enc_out.shape)).to(device)
# #         enc_padding_mask = Variable(torch.rand(enc_padding_mask.shape)).to(device,dtype=torch.long)
# #         context = Variable(torch.rand(context.shape)).to(device)
# #         extra_zeros = Variable(torch.rand(extra_zeros.shape)).to(device)
# #         enc_batch_extend_vocab = Variable(torch.rand(enc_batch_extend_vocab.shape)).to(device)
#         #             sum_temporal_srcs = Variable(torch.rand(sum_temporal_srcs.shape)).to(device)
#         #             prev_s = Variable(torch.rand(prev_s.shape)).to(device)
# #         enc_key_batch = Variable(torch.rand(enc_key_batch.shape)).to(device)
#         enc_key_lens = torch.from_numpy(enc_key_lens).to(device) 
        
#         decoder_writer.add_graph(model.decoder, 
#                          (x_t, s_t, enc_out, enc_padding_mask, context, extra_zeros, enc_batch_extend_vocab, sum_temporal_srcs, prev_s,enc_key_batch, enc_key_lens), verbose=True)
#         decoder_writer.close()
#         break    

In [9]:
# https://blog.csdn.net/u012869752/article/details/72513141
# 由于在jupyter notebook中，args不为空
from glob import glob
# nvidia-smi -pm 1
from transformer_run import *

if __name__ == "__main__":   
    try:
        # --------------------------Training ----------------------------------
        parser = argparse.ArgumentParser()
        parser.add_argument('--train_mle', type=bool, default=True)
        parser.add_argument('--train_rl', type=bool, default=False)
        parser.add_argument('--mle_weight', type=float, default=1.0)
    #         parser.add_argument('--load_model', type=str, default='/0015000_3.29_0.00.tar')
        parser.add_argument('--load_model', type=str, default=None)
        parser.add_argument('--new_lr', type=float, default=None)
        parser.add_argument('--multi_device', type=bool, default=True)
        parser.add_argument('--view', type=bool, default=True)
        parser.add_argument('--pre_train_emb', type=bool, default=True)
        parser.add_argument('--word_emb_type', type=str, default=None)
        parser.add_argument('--train_action', type=bool, default=True)
        opt = parser.parse_args(args=[])

        today = dt.now()
    #         loggerPath = "LOG//%s-(%s_%s_%s)-(%s:%s:%s)"%(opt.word_emb_type,
    #                   today.year,today.month,today.day,
    #                   today.hour,today.minute,today.second)
    #         logger = getLogger(config.loggerName,loggerPath)   

        loggerPath = "LOG//%s"%(opt.word_emb_type)
        logger = getLogger(config.loggerName,loggerPath) 

        if opt.load_model == None:
            shutil.rmtree('runs/Pointer-Generator/Transformer', ignore_errors=True) # clear previous 
            shutil.rmtree('runs/Pointer-Generator/Transformer/exp-4', ignore_errors=True) # clear previous 
            shutil.rmtree('runs/Pointer-Generator/Transformer/Eecoder', ignore_errors=True) # clear previous 
            shutil.rmtree('runs/Pointer-Generator/Transformer/Decoder', ignore_errors=True) # clear previous 

        writer = SummaryWriter('runs/Pointer-Generator/Transformer')
        writer.add_text('Train_Para/',info_str,0)
    #         write_enc_graph()
    #         write_dec_graph()

        if opt.train_action: 
            train_action(opt, logger, writer, train_num)

    except KeyError as e:
        traceback = sys.exc_info()[2]
        print(sys.exc_info())
        print(traceback.tb_lineno)
        print(e)
    finally:
        removeLogger(logger)
        # export scalar data to JSON for external processing
        # tensorboard --logdir /home/eagleuser/Users/leyan/Text-Summarizer-FOP/TensorBoard
#         tensorboard --logdir ./runs
#         if not os.path.exists('TensorBoard'): os.makedirs('TensorBoard')
#         writer.export_scalars_to_json("TensorBoard/test.json")
        writer.close()
        

2020-03-19 21:33:09 - Text-Summary - INFO: - logger已啟動
I0319 21:33:09.378670 139846209922880 <ipython-input-3-00f9962e7fdd>:30] logger已啟動
2020-03-19 21:33:09 - Text-Summary - INFO: - ------Training Setting--------
I0319 21:33:09.383221 139846209922880 transformer_run.py:156] ------Training Setting--------
2020-03-19 21:33:09 - Text-Summary - INFO: - Traing Type :Cameras_new8
I0319 21:33:09.385606 139846209922880 transformer_run.py:158] Traing Type :Cameras_new8
2020-03-19 21:33:09 - Text-Summary - INFO: - Training mle: True, mle weight: 1.00
I0319 21:33:09.386460 139846209922880 transformer_run.py:160] Training mle: True, mle weight: 1.00
2020-03-19 21:33:09 - Text-Summary - INFO: - use pre_train_None vocab_size 50000 

I0319 21:33:09.387500 139846209922880 transformer_run.py:167] use pre_train_None vocab_size 50000 

2020-03-19 21:33:09 - Text-Summary - INFO: - intra_encoder: True intra_decoder: True 

I0319 21:33:09.388511 139846209922880 transformer_run.py:172] intra_encoder: True i

TypeError: forward() takes 3 positional arguments but 4 were given